# Preamble

In [ ]:
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt

In [ ]:
# Whether or not to save matplotlib figures as pdf output file for latex
USE_LATEX_ENGINE = True

if USE_LATEX_ENGINE:
    import matplotlib
    # matplotlib.use("pgf")     # pgf doesn't work for some plots because they exceed the max value that pgf can calculate
    matplotlib.rcParams.update({
        # "pgf.texsystem": "pdflatex",
        # 'pgf.rcfonts': False,
        'font.family': 'serif',
        'text.usetex': True,
    })

# Function Definitions

In [ ]:
def _generate_output_string(element, data):
    """
    Returns a string that contains information about the percentual share of a value in a list
    """
    # if not isinstance(element, str): # make sure element is of type string, convert otherwise
    #     element = str(element)
    
    n_total = len(data)
    bool_list = (data == element)
    n_elements = sum(bool_list)

    return "{:<30}".format(str(n_elements) + '/' + str(n_total) + ' (' + "{:.2f}".format(((n_elements / n_total) * 100)) + '%)')

In [ ]:
def calculate_percentages(normal_data, pneumonia_data, additional_df=None):
    """
    checks if two lists contain the same set of values - returns a warning if the sets dont match - prints a string with information about the distribution if they match
    """
    if len(set(normal_data)) is not len(set(pneumonia_data)):
        print('WARNING! Categories of healthy and pneumonia data are not identical!')
        print('Normal: ' + str(set(normal_data)))
        print('Pneumonia: ' + str(set(pneumonia_data)))
        print('Number of appearences (normal):')
        n_normal_set = ''
        for category in set(normal_data):
            n_normal_set += "'" + str(category) + "': " + str(sum(normal_data == category)) + '\n'   # single quote: non-escaped string, double quote: escaped string
        print(n_normal_set)
        return
    else:
        print(set(normal_data))
    if additional_df is None:
        print('{:<21}'.format('') + '{:<30}'.format('normal') + '{:<30}'.format('pneumonia'))
    else:
        print('{:<21}'.format('') + '{:<30}'.format('normal (total)') + '{:<30}'.format('normal (filtered)') + '{:<30}'.format('pneumonia'))

    for element in set(normal_data):
        category_string = "{:<21}".format(str(element) + ': ')
        if additional_df is None:
            print(category_string + _generate_output_string(element, normal_data) + _generate_output_string(element, pneumonia_data))
        else:
            print(category_string + _generate_output_string(element, normal_data) + _generate_output_string(element, additional_df) + _generate_output_string(element, pneumonia_data))

In [ ]:
def find_appearences(df, column, value):
    """
    searches for appearences of a specified value within the column of a dataframe - prints the number of appearences and the corresponding indexes
    """
    
    value = str(value)
    print('found ' + str(sum(df[column] == value)) + ' rows')
    index_list = df.index[df[column] == value].tolist()
    print('Indexes: ' + str(index_list))
    return index_list

In [ ]:
def drop_from_column(df, column, value):
    """
    searches for appearences of a specified value within a column of a dataframe - drops each row where the value appeared from the dataframe
    """
    value = str(value)
    print('Dropping ' + str(sum(df[column] == value)) + ' rows')
    index_list = df.index[df[column] == value].tolist()
    df.drop(index_list, inplace=True)

# Data preparation

In [ ]:
normal_meta_data = '/mnt/c/Users/Jan/Daten/Geschäftlich/Capgemini/scripts/tmp/filtered_PADCHEST_chest_x_ray_images_labels_160K_01.02.19.csv'
pneumonia_meta_data = '/mnt/c/Users/Jan/Daten/Geschäftlich/Capgemini/scripts/tmp/filtered_pneumonia.csv'

# read csv files, engine='python' improves parsing, column differentiation, etc.
normal_meta_data = pd.read_csv(normal_meta_data, engine='python')
pneumonia_meta_data = pd.read_csv(pneumonia_meta_data, engine='python')

In [ ]:
replace_nan_with = 'U'      # U for Unknown
print('Replacing ' + str(normal_meta_data.isnull().sum().sum()) + ' appearances of NaN with: ' + str(replace_nan_with))
normal_meta_data.fillna(replace_nan_with, inplace=True)   # be careful, this modifies the original dataframe

print('Replacing ' + str(pneumonia_meta_data.isnull().sum().sum()) + ' appearances of NaN with: ' + str(replace_nan_with))
pneumonia_meta_data.fillna(replace_nan_with, inplace=True)   # be careful, this modifies the original dataframe

In [ ]:
qry = ''
qry = qry + 'Labels.str.contains("normal")'
qry = qry + ' & Labels.str.contains("pneumonia")'
print(qry)
res_norm = normal_meta_data.query(qry, engine='python')
res_pneu = pneumonia_meta_data.query(qry, engine='python')

print('Found', len(res_norm), 'overlappings in normal meta data and', len(res_pneu), 'in pneumonia meta data')

In [ ]:
print(len(normal_meta_data))
normal_meta_data.drop(res_norm.index, inplace=True)
print(len(normal_meta_data))

print(len(pneumonia_meta_data))
pneumonia_meta_data.drop(res_pneu.index, inplace=True)
print(len(pneumonia_meta_data))

In [ ]:
FIRST_GROUP_BOUNDARY = 12
SECOND_GROUP_BOUNDARY = 62

drop_from_column(normal_meta_data, 'PatientBirth', 'U') # drop unknown birth years, since there's only one entry

# This is clearly not the fastest way to do this, as using df.loc in loops is not recommended
for df in [normal_meta_data, pneumonia_meta_data]:
    i = 0
    for row in df.itertuples():
        age = int(str(row.StudyDate_DICOM)[:4]) - int(row.PatientBirth)
        df.loc[row.Index, 'Age'] = age

        if 0 <= age <= FIRST_GROUP_BOUNDARY:
            df.loc[row.Index, 'AgeGroup'] = int(1)
        elif FIRST_GROUP_BOUNDARY < age <= SECOND_GROUP_BOUNDARY:
            df.loc[row.Index, 'AgeGroup'] = int(2)
        elif SECOND_GROUP_BOUNDARY < age:
            df.loc[row.Index, 'AgeGroup'] = int(3)
        else:
            raise ValueError("Calculated age was either below 0 or a non numerical type")

        if str(row.Projection) == 'AP_horizontal':
            i += 1
            df.loc[row.Index, 'ProjectionSimplified'] = 'AP'
        else:
            df.loc[row.Index, 'ProjectionSimplified'] = row.Projection
    print('Replaced AP_horizontal with AP', i, 'times')


# Data Analysis

In [ ]:
print('==== Distribution of projections ====\n')

calculate_percentages(normal_meta_data['Projection'], pneumonia_meta_data['Projection'])

find_appearences(normal_meta_data, 'Projection', 'UNK')
drop_from_column(normal_meta_data, 'Projection', 'UNK')

drop_from_column(normal_meta_data, 'Projection', 'COSTAL')
drop_from_column(pneumonia_meta_data, 'Projection', 'COSTAL')

calculate_percentages(normal_meta_data['Projection'], pneumonia_meta_data['Projection'])

In [ ]:
calculate_percentages(normal_meta_data['ProjectionSimplified'], pneumonia_meta_data['ProjectionSimplified'])
calculate_percentages(normal_meta_data['AgeGroup'], pneumonia_meta_data['AgeGroup'])

In [ ]:
print('==== Distribution of MethodLabel ====\n')

calculate_percentages(normal_meta_data['MethodLabel'], pneumonia_meta_data['MethodLabel'])

In [ ]:
print('==== Distribution of Patient Sex ====\n')

calculate_percentages(normal_meta_data['PatientSex_DICOM'], pneumonia_meta_data['PatientSex_DICOM'])


In [ ]:
calculate_percentages(normal_meta_data['PatientSex_DICOM'], pneumonia_meta_data['PatientSex_DICOM'])

drop_from_column(normal_meta_data, 'PatientSex_DICOM', 'U')     # drop 'U' and 'O' since they are heavily underrepresented
drop_from_column(normal_meta_data, 'PatientSex_DICOM', 'O')

calculate_percentages(normal_meta_data['PatientSex_DICOM'], pneumonia_meta_data['PatientSex_DICOM'])

In [ ]:
print('==== Distribution of ExposureTime ====\n')

calculate_percentages(normal_meta_data['ExposureTime'], pneumonia_meta_data['ExposureTime'])


In [ ]:
calculate_percentages(normal_meta_data['Exposure_DICOM'], pneumonia_meta_data['Exposure_DICOM'])

filtered_exposure_normal = [int(x) for x in normal_meta_data['Exposure_DICOM'] if str(x) != 'None']
filtered_exposure_pneumonia = [int(x) for x in pneumonia_meta_data['Exposure_DICOM'] if str(x) != 'None']
print(len(filtered_exposure_normal))
print(len(filtered_exposure_pneumonia))


In [ ]:
filtered_exposure_normal = sorted(filtered_exposure_normal)
filtered_exposure_pneumonia = sorted(filtered_exposure_pneumonia)

fig, ax_exp_time = plt.subplots()  # a figure with a single Axes

ax_exp_time.set_yscale('log')   # logarithmic scale on y axis

# bins must be -0.5 to center the x ticks, for detailled explanation see https://stackoverflow.com/questions/27083051/matplotlib-xticks-not-lining-up-with-histogram
ax_exp_time.hist(filtered_exposure_normal, np.arange(42)-0.5, alpha=0.5, label="Normal")
ax_exp_time.hist(filtered_exposure_pneumonia, np.arange(42)-0.5, alpha=0.5, color='r', label="Pneumonie")
ax_exp_time.set_xlabel('Strom * Zeit in mAs')
ax_exp_time.set_ylabel('Anzahl Röntgenbilder')
ax_exp_time.legend()
ax_exp_time.grid(True)

if USE_LATEX_ENGINE:
    plt.savefig("/mnt/c/Users/Jan/Daten/Dropbox/Master/3_Semester/Masterarbeit/Latex/python_output/padchest_combined_exposure.pdf")

In [ ]:
exposure_percentage_normal = [(filtered_exposure_normal.count(x)*100) / len(filtered_exposure_normal) for x in np.arange(42)]
exposure_percentage_pneumonia = [(filtered_exposure_pneumonia.count(x)*100) / len(filtered_exposure_pneumonia) for x in np.arange(42)]

fig, ax = plt.subplots()  # a figure with a single Axes

# bins must be -0.5 to center the x ticks, for detailled explanation see https://stackoverflow.com/questions/27083051/matplotlib-xticks-not-lining-up-with-histogram
ax.bar(np.arange(6)-0.125, exposure_percentage_normal[:6], width=0.25, label="Normal")
ax.bar(np.arange(6)+0.125, exposure_percentage_pneumonia[:6], color='r', width=0.25, label="Pneumonie")
ax.set_xlabel('Strom * Zeit in mAs')
ax.set_ylabel('Prozentualer Anteil')
ax.legend()
ax.grid(True)

if USE_LATEX_ENGINE:
    plt.savefig("/mnt/c/Users/Jan/Daten/Dropbox/Master/3_Semester/Masterarbeit/Latex/python_output/padchest_combined_exposure_percentage.pdf")

In [ ]:
print('==== Distribution of XRayTubeCurrent_DICOM ====\n')

calculate_percentages(normal_meta_data['XRayTubeCurrent_DICOM'], pneumonia_meta_data['XRayTubeCurrent_DICOM'])

In [ ]:
print('==== Distribution of Exposure_DICOM ====\n')

calculate_percentages(normal_meta_data['Exposure_DICOM'], pneumonia_meta_data['Exposure_DICOM'])


In [ ]:
print('==== Distribution of Modality_DICOM ====\n')

calculate_percentages(normal_meta_data['Modality_DICOM'], pneumonia_meta_data['Modality_DICOM'])

In [ ]:
print('==== Distribution of Manufacturer_DICOM ====\n')

calculate_percentages(normal_meta_data['Manufacturer_DICOM'], pneumonia_meta_data['Manufacturer_DICOM'])

In [ ]:
print('==== Distribution of PixelAspectRatio_DICOM ====\n')
calculate_percentages(normal_meta_data['PixelAspectRatio_DICOM'], pneumonia_meta_data['PixelAspectRatio_DICOM'])

In [ ]:
print('==== Distribution of Age ====\n')
calculate_percentages(normal_meta_data['PatientBirth'], pneumonia_meta_data['PatientBirth'])

In [ ]:
normal_age_list = list()
number_of_normal = list()
for y in set(normal_meta_data['PatientBirth']):
    normal_age_list += [int(y)]
    number_of_normal += [sum(normal_meta_data['PatientBirth'] == y)]

print(normal_age_list)
print(number_of_normal)

In [ ]:
pneumonia_age_list = list()
number_of_pneumonia = list()
for y in set(pneumonia_meta_data['PatientBirth']):
    pneumonia_age_list += [int(y)]
    number_of_pneumonia += [sum(pneumonia_meta_data['PatientBirth'] == y)]

print(pneumonia_age_list)
print(number_of_pneumonia)

In [ ]:
plt.axis([1900, 2021 , 0, 1001])
plt.plot(normal_age_list, number_of_normal, label="Normal")
plt.plot(pneumonia_age_list, number_of_pneumonia, 'r', label="Pneumonie")
plt.ylabel('Anzahl Patienten')
plt.xlabel('Geburtsjahr')
plt.legend()
plt.grid(True)
# plt.show()

if USE_LATEX_ENGINE:
    plt.savefig('/mnt/c/Users/Jan/Daten/Dropbox/Master/3_Semester/Masterarbeit/Latex/python_output/year_of_birth_diagram.pdf')


In [ ]:
maximum_age = max(max(set(normal_meta_data['Age'])), max(set(pneumonia_meta_data['Age'])))
x_axis_age = np.arange(maximum_age + 2)-0.5 # would be +1 for right aligned, but we want to center the x ticks -> +0.5
plt.hist(normal_meta_data['Age'], x_axis_age, alpha=0.5, label="Normal")
plt.hist(pneumonia_meta_data['Age'], x_axis_age, alpha=0.5, color='r', label="Pneumonie")
plt.ylabel('Anzahl Patienten')
plt.xlabel('Alter')
plt.legend()
plt.grid(True)
# plt.show()

if USE_LATEX_ENGINE:
    plt.savefig('/mnt/c/Users/Jan/Daten/Dropbox/Master/3_Semester/Masterarbeit/Latex/python_output/padchest_age_histogram.pdf')


In [ ]:
set(normal_meta_data)

In [ ]:
duplicate_patients = set(normal_meta_data['PatientID']) & set(pneumonia_meta_data['PatientID'])
print(str(len(duplicate_patients)) + ' Patients are both in Normal and Pneumonia data present')
print('Total number of Patients:\nNormal: ' + str(len(set(normal_meta_data['PatientID']))) + '\nPneumonia: ' + str(len(set(pneumonia_meta_data['PatientID']))))

In [ ]:
qry = ''
first = True
for id in list(duplicate_patients):
    if first:
        qry = qry + 'PatientID == \'' + str(id) + '\''
        first = False
    else:
        qry = qry + ' | PatientID == \'' + str(id) + '\''
duplicate_df = normal_meta_data.query(qry)

In [ ]:
calculate_percentages(normal_meta_data['ProjectionSimplified'], pneumonia_meta_data['ProjectionSimplified'])
calculate_percentages(duplicate_df['ProjectionSimplified'], pneumonia_meta_data['ProjectionSimplified'])

# Filtering for final Dataset

## Patients who have images with and without pneumonia
All samples of these patients labeled with normal + projection PA will be removed from the dataset, to make sure their "normal" images don't show a light pneumonia.<br>
normal label + AP will be kept, as the combination of AP + normal is rare

In [ ]:
print("Dropping", sum(duplicate_df['ProjectionSimplified'] == 'PA'), "samples labeled as normal + projection PA whose patients also have samples with pneumonia label")

print(len(normal_meta_data))
normal_meta_data.drop(duplicate_df.query("ProjectionSimplified == 'PA'").index, inplace=True)
print(len(normal_meta_data))



In [ ]:
calculate_percentages(duplicate_df.query("ProjectionSimplified == 'AP'")['MethodLabel'], pneumonia_meta_data['MethodLabel'])
calculate_percentages(normal_meta_data['MethodLabel'], pneumonia_meta_data['MethodLabel'])

# Experimantal random sampling

In [ ]:

filtered_normal_meta = []
copy_normal_meta_imagingdynamics = normal_meta_data.loc[normal_meta_data['Manufacturer_DICOM'] == "ImagingDynamicsCompanyLtd"]
copy_normal_meta_philips = normal_meta_data.loc[normal_meta_data['Manufacturer_DICOM'] == "PhilipsMedicalSystems"]

random_imagingdynamics = copy_normal_meta_imagingdynamics.sample(n = 1114)
random_philips = copy_normal_meta_philips.sample(n = 4106)
random_normal = random_imagingdynamics.append(random_philips)
print(len(random_imagingdynamics))
print(len(random_philips))
print(len(random_normal))


In [ ]:
print('==== Distribution of projections ====\n')
calculate_percentages(normal_meta_data['Projection'], pneumonia_meta_data['Projection'], random_normal['Projection'])

In [ ]:
find_appearences(normal_meta_data, 'Projection', 'COSTAL')

In [ ]:

filtered_normal_meta = []
copy_normal_meta_imagingdynamics = normal_meta_data.loc[normal_meta_data['Manufacturer_DICOM'] == "ImagingDynamicsCompanyLtd"]
copy_normal_meta_philips = normal_meta_data.loc[normal_meta_data['Manufacturer_DICOM'] == "PhilipsMedicalSystems"]

random_imagingdynamics = copy_normal_meta_imagingdynamics.sample(n = 1116)
random_philips = copy_normal_meta_philips.sample(n = 4107)
random_normal = random_imagingdynamics.append(random_philips)

print("ImagingDynamics: " + str(len(random_imagingdynamics)) + '/' + str(len(copy_normal_meta_imagingdynamics)))
print("Philips; " + str(len(random_philips)) + '/' + str(len(copy_normal_meta_philips)))
print(len(random_normal))

# random_normal = normal_meta_data.sample(n = len(pneumonia_meta_data))

print('==== Distribution of projections ====\n')
calculate_percentages(normal_meta_data['Projection'], pneumonia_meta_data['Projection'], random_normal['Projection'])

In [ ]:
print('==== Distribution of Manufacturers ====\n')
calculate_percentages(normal_meta_data['Manufacturer_DICOM'], pneumonia_meta_data['Manufacturer_DICOM'], random_normal['Manufacturer_DICOM'])

In [ ]:
print('==== Distribution of MethodLabel ====\n')
calculate_percentages(random_normal['MethodLabel'], pneumonia_meta_data['MethodLabel'])

In [ ]:
print('==== Distribution of Patient Sex ====\n')
calculate_percentages(random_normal['PatientSex_DICOM'], pneumonia_meta_data['PatientSex_DICOM'])

In [ ]:
print('==== Distribution of Modality_DICOM ====\n')
calculate_percentages(random_normal['Modality_DICOM'], pneumonia_meta_data['Modality_DICOM'])

In [ ]:
print('==== Distribution of Manufacturer_DICOM ====\n')
calculate_percentages(random_normal['Manufacturer_DICOM'], pneumonia_meta_data['Manufacturer_DICOM'])

In [ ]:
print('==== Distribution of MethodLabel ====\n')
calculate_percentages(random_normal['MethodLabel'], pneumonia_meta_data['MethodLabel'])

In [ ]:
print('==== Distribution of MethodProjection ====\n')
calculate_percentages(random_normal['MethodProjection'], pneumonia_meta_data['MethodProjection'])

# Stratified Sampling from Kaggle

In [ ]:
'''
This module contains functions that computes stratified sampling of pandas dataframes.
'''
# Required libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Functions

In [ ]:
def __smpl_size(population, size):
    '''
    A function to compute the sample size. If not informed, a sampling 
    size will be calculated using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2

        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error

        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)

        where:
            - cochran_n = result of the previous formula
            - N is the population size
    Parameters
    ----------
        :population: population size
        :size: sample size (default = None)
    Returns
    -------
    Calculated sample size to be used in the functions:
        - stratified_sample
        - stratified_sample_report
    '''
    if size is None:
        cochran_n = round(((1.96)**2 * 0.5 * 0.5)/ 0.02**2)
        n = round(cochran_n/(1+((cochran_n -1) /population)))
    elif size >= 0 and size < 1:
        n = round(population * size)
    elif size < 0:
        raise ValueError('Parameter "size" must be an integer or a proportion between 0 and 0.99.')
    elif size >= 1:
        n = size
    return n

In [ ]:
def stratified_sample_report(df, strata, size=None):
    '''
    Generates a dataframe reporting the counts in each stratum and the counts
    for the final sampled dataframe.

    Parameters
    ----------
    :df: pandas dataframe from which data will be sampled.
    :strata: list containing columns that will be used in the stratified sampling.
    :size: sampling size. If not informed, a sampling size will be calculated
        using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2

        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error

        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)

        where:
            - cochran_n = result of the previous formula
            - N is the population size

    Returns
    -------
    A dataframe reporting the counts in each stratum and the counts
    for the final sampled dataframe.
    '''
    population = len(df)
    size = __smpl_size(population, size)
    tmp = df[strata]
    tmp['size'] = 1
    tmp_grpd = tmp.groupby(strata).count().reset_index()
    tmp_grpd['samp_size'] = round(size/population * tmp_grpd['size']).astype(int)
    return tmp_grpd


In [ ]:
def stratified_sample(df, strata, size=None, seed=None, keep_index= True):
    '''
    It samples data from a pandas dataframe using strata. These functions use
    proportionate stratification:
    n1 = (N1/N) * n
    where:
        - n1 is the sample size of stratum 1
        - N1 is the population size of stratum 1
        - N is the total population size
        - n is the sampling size
    Parameters
    ----------
    :df: pandas dataframe from which data will be sampled.
    :strata: list containing columns that will be used in the stratified sampling.
    :size: sampling size. If not informed, a sampling size will be calculated
        using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2
        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error
        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)
        where:
            - cochran_n = result of the previous formula
            - N is the population size
    :seed: sampling seed
    :keep_index: if True, it keeps a column with the original population index indicator
    
    Returns
    -------
    A sampled pandas dataframe based in a set of strata.
    Examples
    --------
    >> df.head()
    	id  sex age city 
    0	123 M   20  XYZ
    1	456 M   25  XYZ
    2	789 M   21  YZX
    3	987 F   40  ZXY
    4	654 M   45  ZXY
    ...
    # This returns a sample stratified by sex and city containing 30% of the size of
    # the original data
    >> stratified = stratified_sample(df=df, strata=['sex', 'city'], size=0.3)
    Requirements
    ------------
    - pandas
    - numpy
    '''
    population = len(df)
    size = __smpl_size(population, size)
    tmp = df[strata]
    tmp['size'] = 1
    tmp_grpd = tmp.groupby(strata).count().reset_index()
    tmp_grpd['samp_size'] = round(size/population * tmp_grpd['size']).astype(int)

    # controlling variable to create the dataframe or append to it
    first = True 
    for i in range(len(tmp_grpd)):
        # query generator for each iteration
        qry=''
        for s in range(len(strata)):
            stratum = strata[s]
            value = tmp_grpd.iloc[i][stratum]
            n = tmp_grpd.iloc[i]['samp_size']

            if type(value) == str:
                value = "'" + str(value) + "'"
            
            if s != len(strata)-1:
                qry = qry + stratum + ' == ' + str(value) +' & '
            else:
                qry = qry + stratum + ' == ' + str(value)
        
        # final dataframe
        if first:
            stratified_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            first = False
        else:
            tmp_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            stratified_df = stratified_df.append(tmp_df, ignore_index=True)
    
    return stratified_df


In [ ]:
def stratified_sample_report(df, strata, size=None):
    '''
    Generates a dataframe reporting the counts in each stratum and the counts
    for the final sampled dataframe.

    Parameters
    ----------
    :df: pandas dataframe from which data will be sampled.
    :strata: list containing columns that will be used in the stratified sampling.
    :size: sampling size. If not informed, a sampling size will be calculated
        using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2

        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error

        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)

        where:
            - cochran_n = result of the previous formula
            - N is the population size

    Returns
    -------
    A dataframe reporting the counts in each stratum and the counts
    for the final sampled dataframe.
    '''
    population = len(df)
    size = __smpl_size(population, size)
    tmp = df[strata]
    tmp['size'] = 1
    tmp_grpd = tmp.groupby(strata).count().reset_index()
    tmp_grpd['samp_size'] = round(size/population * tmp_grpd['size']).astype(int)
    return tmp_grpd

In [ ]:
def stratified_sample_transferred(df_target, df_proportion, strata, size=None, seed=None, keep_index= True):
    '''
    It samples data from a pandas dataframe using strata. These functions use
    proportionate stratification:
    n1 = (N1/N) * n
    where:
        - n1 is the sample size of stratum 1
        - N1 is the population size of stratum 1
        - N is the total population size
        - n is the sampling size
    Parameters
    ----------
    :df_target: pandas dataframe from which data will be sampled.
    :df_proportion: pandas dataframe from which the proportions for sampling will be used.
    :strata: list containing columns that will be used in the stratified sampling.
    :size: sampling size. If not informed, a sampling size will be calculated
        using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2
        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error
        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)
        where:
            - cochran_n = result of the previous formula
            - N is the population size
    :seed: sampling seed
    :keep_index: if True, it keeps a column with the original population index indicator
    
    Returns
    -------
    A sampled pandas dataframe based in a set of strata.
    Examples
    --------
    >> df.head()
    	id  sex age city 
    0	123 M   20  XYZ
    1	456 M   25  XYZ
    2	789 M   21  YZX
    3	987 F   40  ZXY
    4	654 M   45  ZXY
    ...
    # This returns a sample stratified by sex and city containing 30% of the size of
    # the original data
    >> stratified = stratified_sample(df=df, strata=['sex', 'city'], size=0.3)
    Requirements
    ------------
    - pandas
    - numpy
    '''
    
    # population = len(df)
    # size = __smpl_size(population, size)
    # tmp = df[strata]
    # tmp['size'] = 1
    # tmp_grpd = tmp.groupby(strata).count().reset_index()
    # tmp_grpd['samp_size'] = round(size/population * tmp_grpd['size']).astype(int)


    population = len(df_proportion)
    size = __smpl_size(population, size)

    tmp_proportion = df_proportion[strata]
    tmp = df_target[strata]

    tmp_proportion['size'] = 1
    tmp['size'] = 1

    tmp_proportion_grpd = tmp_proportion.groupby(strata).count().reset_index()
    tmp_grpd = tmp.groupby(strata).count().reset_index()

    if not tmp_grpd[strata].equals(tmp_proportion_grpd[strata]):
        print(tmp_grpd)
        print(tmp_proportion_grpd)
        raise ValueError("Dataframes don't have the same groups for the given set of stratas. You can check the differences in the terminal output.")        

    tmp_proportion_grpd['samp_size'] = round(size/population * tmp_proportion_grpd['size']).astype(int)
    tmp_grpd['samp_size'] = round(size/population * tmp_proportion_grpd['size']).astype(int)

    # controlling variable to create the dataframe or append to it
    first = True
    # Hier muss nichts geändert werden, da die Gruppen ohnehin die gleichen sein sollten -> len(tmp_grpd) == len(tmp_proportion_grpd)
    for i in range(len(tmp_grpd)):
        # query generator for each iteration
        qry=''
        for s in range(len(strata)):
            stratum = strata[s]
            # Hier wird n berechnet, tmp_proportion_grpd statt tmp_grpd
            
            value = tmp_grpd.iloc[i][stratum]
            n = tmp_grpd.iloc[i]['samp_size']

            if type(value) == str:
                value = "'" + str(value) + "'"
            
            if s != len(strata)-1:
                qry = qry + stratum + ' == ' + str(value) +' & '
            else:
                qry = qry + stratum + ' == ' + str(value)
        
        # query dataframe
        queried_df = df_target.query(qry)

        # check if result contains enough samples / rows
        if len(queried_df) < n:
            print("Warning! The required number of samples (" + str(n) + ") could not be retrieved.")
            print("Using all available entries (" + str(len(queried_df)) + ") of group:")
            print(qry)
            n = len(queried_df)

        # sample dataframe
        sampled_df = queried_df.sample(n=n, random_state=seed).reset_index(drop=(not keep_index))

        # assign to final dataframe
        if first:
            stratified_df = sampled_df
            first = False
        else:
            stratified_df = stratified_df.append(sampled_df, ignore_index=True)
    
    return stratified_df

In [ ]:
stratified_sample_report(normal_meta_data, ['Manufacturer_DICOM', 'Projection', 'PatientSex_DICOM'], size=2000)


In [ ]:
indexes = find_appearences(normal_meta_data, 'Projection', 'AP_horizontal')

In [ ]:
# import matplotlib.image as mpimg
# import random

# fig_nr = 1

# for i in random.sample(indexes, 10):
#     plt.figure(fig_nr)
#     fig_nr += 1
#     img = mpimg.imread('/mnt/f/BIMCV-PadChest/unzipped/' + str(normal_meta_data.loc[i]['ImageDir']) + '/' + str(normal_meta_data.loc[i]['ImageID']))
#     imgplot = plt.imshow(img, cmap="gray")
# plt.show()

In [ ]:
stratified_sample_report(normal_meta_data, ['Manufacturer_DICOM', 'Modality_DICOM', 'PatientSex_DICOM', 'ProjectionSimplified', 'AgeGroup'])

In [ ]:
stratified_sample_report(pneumonia_meta_data, ['Manufacturer_DICOM', 'Modality_DICOM', 'PatientSex_DICOM', 'ProjectionSimplified', 'AgeGroup'])

In [ ]:
filtered_df = stratified_sample_transferred(normal_meta_data, pneumonia_meta_data, ['Manufacturer_DICOM', 'Modality_DICOM', 'PatientSex_DICOM', 'ProjectionSimplified', 'MethodProjection'], size=len(pneumonia_meta_data))

print('\nFiltered dataframe has', len(filtered_df), 'samples')

In [ ]:
maximum_age = max(max(set(filtered_df['Age'])), max(set(pneumonia_meta_data['Age'])))
x_axis_age = np.arange(maximum_age + 2)-0.5 # would be +1 for right aligned, but we want to center the x ticks -> +0.5

plt.hist(filtered_df['Age'], x_axis_age, alpha=0.5, label="Normal")
plt.hist(pneumonia_meta_data['Age'], x_axis_age, alpha=0.5, color='r', label="Pneumonie")
plt.ylabel('Anzahl Patienten')
plt.xlabel('Alter')
plt.legend()
plt.grid(True)

if USE_LATEX_ENGINE:
    plt.savefig('/mnt/c/Users/Jan/Daten/Dropbox/Master/3_Semester/Masterarbeit/Latex/python_output/padchest_age_histogram_no_groups.pdf')

In [ ]:
total_error = 0
for age in np.arange(max(max(filtered_df['Age']), max(pneumonia_meta_data['Age']))):
    error = abs(list(filtered_df['Age']).count(age) - list(pneumonia_meta_data['Age']).count(age))
    # print(error)
    total_error += error
print('Total error was:', total_error)

In [ ]:
qry = "(Manufacturer_DICOM == 'ImagingDynamicsCompanyLtd' & Modality_DICOM == 'CR' & PatientSex_DICOM == 'F' & ProjectionSimplified == 'AP' & AgeGroup == '3.0')"
qry = qry + " | (Manufacturer_DICOM == 'PhilipsMedicalSystems' & Modality_DICOM == 'DX' & PatientSex_DICOM == 'F' & ProjectionSimplified == 'AP' & AgeGroup == '2.0')"
qry = qry + " | (Manufacturer_DICOM == 'PhilipsMedicalSystems' & Modality_DICOM == 'DX' & PatientSex_DICOM == 'M' & ProjectionSimplified == 'AP' & AgeGroup == '2.0')"
dropped_samples = normal_meta_data.query(qry)
normal_meta_data.drop(dropped_samples.index, inplace=True)

print(len(dropped_samples), 'samples were temporarily removed')

In [ ]:
filtered_df = stratified_sample_transferred(normal_meta_data, pneumonia_meta_data, ['Manufacturer_DICOM', 'Modality_DICOM', 'PatientSex_DICOM', 'ProjectionSimplified', 'AgeGroup'], size=len(pneumonia_meta_data) + 174) # 174 was determined empirically so the length of the filtered dataframe + all remaining ap samples matches the length of pneumonia data
# missing values: 'MethodLabel'

In [ ]:
len(filtered_df)

In [ ]:
stratified_sample_report(pneumonia_meta_data, ['Manufacturer_DICOM', 'Modality_DICOM', 'PatientSex_DICOM', 'AgeGroup', 'ProjectionSimplified'], size=len(pneumonia_meta_data))

In [ ]:
maximum_age = max(max(set(filtered_df['Age'])), max(set(pneumonia_meta_data['Age'])))
x_axis_age = np.arange(maximum_age + 2)-0.5 # would be +1 for right aligned, but we want to center the x ticks -> +0.5


plt.hist(filtered_df['Age'], x_axis_age, alpha=0.5, label="Normal")
plt.hist(pneumonia_meta_data['Age'], x_axis_age, alpha=0.5, color='r', label="Pneumonie")
plt.axvline(x=FIRST_GROUP_BOUNDARY + 0.5, label='Altersgrenzen', c='k', linestyle='--', linewidth=1)
plt.axvline(x=SECOND_GROUP_BOUNDARY + 0.5, c='k', linestyle='--', linewidth=1)
plt.ylabel('Anzahl Patienten')
plt.xlabel('Alter')

handles, labels = plt.gca().get_legend_handles_labels()
order = [1,2,0]     # order in the array determines position, number in the array is the original order
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order])

plt.grid(True)


if USE_LATEX_ENGINE:
    plt.savefig('/mnt/c/Users/Jan/Daten/Dropbox/Master/3_Semester/Masterarbeit/Latex/python_output/padchest_age_histogram_with_groups.pdf')

In [ ]:
total_error = 0
for age in np.arange(max(max(filtered_df['Age']), max(pneumonia_meta_data['Age']))):
    error = abs(list(filtered_df['Age']).count(age) - list(pneumonia_meta_data['Age']).count(age))
    # print(error)
    total_error += error
print('Total error was:', total_error)



In [ ]:
calculate_percentages(normal_meta_data['ProjectionSimplified'], pneumonia_meta_data['ProjectionSimplified'], filtered_df['ProjectionSimplified'])

In [ ]:
remaining_ap_df = normal_meta_data.copy()
remaining_ap_df.drop(filtered_df['index'], inplace=True)

In [ ]:
remaining_ap = remaining_ap_df.query("ProjectionSimplified == 'AP'")
print(len(remaining_ap))
filtered_df = filtered_df.append(remaining_ap)

In [ ]:
calculate_percentages(filtered_df['MethodLabel'], pneumonia_meta_data['MethodLabel'])
calculate_percentages(filtered_df['MethodProjection'], pneumonia_meta_data['MethodProjection'])
calculate_percentages(filtered_df['Manufacturer_DICOM'], pneumonia_meta_data['Manufacturer_DICOM'])
calculate_percentages(filtered_df['PatientSex_DICOM'], pneumonia_meta_data['PatientSex_DICOM'])


In [ ]:
calculate_percentages(filtered_df['BitsStored_DICOM'], pneumonia_meta_data['BitsStored_DICOM'])
calculate_percentages(filtered_df['SpatialResolution_DICOM'], pneumonia_meta_data['SpatialResolution_DICOM'])


In [ ]:
calculate_percentages(filtered_df['ViewPosition_DICOM'], pneumonia_meta_data['ViewPosition_DICOM'])